In [ ]:

import pandas as pd
from selenium import webdriver
import time
from selenium.common.exceptions import NoSuchElementException
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By
from selenium.webdriver.common.action_chains import ActionChains
import re
from selenium.common.exceptions import StaleElementReferenceException
from selenium.common.exceptions import StaleElementReferenceException, NoSuchElementException, NoSuchWindowException

from selenium.webdriver.common.by import By

In [ ]:
chrome_options = webdriver.ChromeOptions()
prefs = {"profile.default_content_setting_values.notifications": 2}
chrome_options.add_experimental_option("prefs", prefs)

driver = webdriver.Chrome(options=chrome_options)
driver.get('https://www.imdb.com/search/title/?moviemeter=1001,1500')

In [ ]:
# Hàm lọc
def filter_array(array_elements):
    array = []
    for element in array_elements:
        array_processed = element.text.replace('\n', '').strip()
        array.append(array_processed)
    return array

# Đợi cho phần tử "See more" hiển thị
wait = WebDriverWait(driver, 10)
see_more_button = wait.until(EC.visibility_of_element_located((By.XPATH, "//button[contains(@class, 'ipc-see-more__button')]")))

# Cuộn trang web xuống tới cuối
while see_more_button.is_displayed():
    actions = ActionChains(driver)
    actions.move_to_element(see_more_button).perform()
    try:
        driver.execute_script("arguments[0].click();", see_more_button)
        see_more_button = wait.until(EC.visibility_of_element_located((By.XPATH, "//button[contains(@class, 'ipc-see-more__button')]")))
    except:
        break


NoSuchWindowException: Message: no such window: target window already closed
from unknown error: web view not found
  (Session info: chrome=130.0.6723.117)
Stacktrace:
	GetHandleVerifier [0x00007FF6D1D938A5+3004357]
	(No symbol) [0x00007FF6D1A29970]
	(No symbol) [0x00007FF6D18D582A]
	(No symbol) [0x00007FF6D18AFA75]
	(No symbol) [0x00007FF6D1952C77]
	(No symbol) [0x00007FF6D196B351]
	(No symbol) [0x00007FF6D194B983]
	(No symbol) [0x00007FF6D1917628]
	(No symbol) [0x00007FF6D1918791]
	GetHandleVerifier [0x00007FF6D1DBA00D+3161901]
	GetHandleVerifier [0x00007FF6D1E0E060+3506048]
	GetHandleVerifier [0x00007FF6D1E0400D+3465005]
	GetHandleVerifier [0x00007FF6D1B80EEB+830987]
	(No symbol) [0x00007FF6D1A3467F]
	(No symbol) [0x00007FF6D1A309D4]
	(No symbol) [0x00007FF6D1A30B6D]
	(No symbol) [0x00007FF6D1A20149]
	BaseThreadInitThunk [0x00007FF8B09B259D+29]
	RtlUserThreadStart [0x00007FF8B2D0AF38+40]


In [ ]:
# Lọc movie link
movie_link_elements = driver.find_elements(By.CSS_SELECTOR,'a.ipc-title-link-wrapper')
movie_links = [element.get_attribute('href') for element in movie_link_elements]

movies = []
posters = []
release_date = []
film_actors = []
duration = []
country = []
companies = []
movie_category = []
ratings = []
directors = []
certificate = []
budget= []
boxoffice= []
original_language = []
taglines = []
writing_credits = []
description = []

In [ ]:
for movie_link in movie_links:
    driver.get(movie_link) # Open the movie URL

    # Allow time for the page to load
    time.sleep(3)

    # Scrape the information
    # Lọc movie
    try:
        movie = driver.find_element('css selector', 'h1[data-testid="hero__pageTitle"] span.hero__primary-text')
        if movie:
            movies.append(movie.text)
        else:
            movies.append(None)
    except:
        movies.append(None)
    print('Movies:', movie.text)
    # Lọc description
    try:
        description_element = driver.find_element(By.CSS_SELECTOR, "p[data-testid='plot']")  # CSS selector for the plot
        description_text = description_element.text
        description.append(description_text)
    except NoSuchElementException:
        description.append(None)
    #Lọc ratings
    try:
        rating_element = driver.find_element(By.CSS_SELECTOR, "span.sc-d541859f-1.imUuxf") #div[data-testid='hero-rating-bar__aggregate-rating__score'] span
        imdb_rating = rating_element.text  # Extract the text (e.g., "8.8")
        ratings.append(imdb_rating)
    except NoSuchElementException:
        ratings.append(None)
    print('Ratings:',imdb_rating)
    #Lọc Certificate
    certificate_element = None
    try:
        certificate_element = driver.find_element(By.XPATH, '//a[contains(@href, "/parentalguide/certificates")]')
    except NoSuchElementException:
        pass

    certificate_text = certificate_element.text if certificate_element else None
    certificate.append(certificate_text)
    print('Certificate:',certificate_text)
    # Lọc diễn viên
    actor_elements = driver.find_elements(By.CSS_SELECTOR,'a[data-testid="title-cast-item__actor"]')
    actor_text = ', '.join([element.text for element in actor_elements])
    film_actors.append(actor_text)
    # Lọc poster
    try:
        poster_element = driver.find_element(By.CSS_SELECTOR, "div[data-testid='hero-media__poster'] img")
        poster_url = poster_element.get_attribute("src")  # Lấy link từ thuộc tính 'src'
        posters.append(poster_url)
    except NoSuchElementException:
        posters.append(None)
    print('Poster URL:', poster_url)
    # Lọc duration
    duration_element = None
    try:
        duration_element = driver.find_element(By.CSS_SELECTOR,'li[data-testid="title-techspec_runtime"] div.ipc-metadata-list-item__content-container')
    except NoSuchElementException:
        pass
    duration_text = duration_element.text if duration_element else None
    duration.append(duration_text)

    # Lọc release date
    release_date_element = None
    try:
        release_date_element = driver.find_element(By.CSS_SELECTOR,'li[data-testid="title-details-releasedate"] div.ipc-metadata-list-item__content-container ul.ipc-inline-list.ipc-inline-list--show-dividers.ipc-inline-list--inline.ipc-metadata-list-item__list-content.base li.ipc-inline-list__item a.ipc-metadata-list-item__list-content-item.ipc-metadata-list-item__list-content-item--link')
    except NoSuchElementException:
        pass
    release_date_text = release_date_element.text if release_date_element else None
    release_date.append(release_date_text)
    # Lọc country
    country_element = None
    try:
        country_element = driver.find_element(By.CSS_SELECTOR,'li[data-testid="title-details-origin"] div.ipc-metadata-list-item__content-container ul.ipc-inline-list.ipc-inline-list--show-dividers.ipc-inline-list--inline.ipc-metadata-list-item__list-content.base li.ipc-inline-list__item a.ipc-metadata-list-item__list-content-item.ipc-metadata-list-item__list-content-item--link')
    except NoSuchElementException:
        pass
    country_text = country_element.text if country_element else None
    country.append(country_text)
    # Lọc budget và box office
    budget_element = None
    boxoffice_element = None
    try:
        budget_element = driver.find_element(By.CSS_SELECTOR,'li[data-testid="title-boxoffice-budget"] div.ipc-metadata-list-item__content-container ul.ipc-inline-list.ipc-inline-list--show-dividers.ipc-inline-list--inline.ipc-metadata-list-item__list-content.base li.ipc-inline-list__item span.ipc-metadata-list-item__list-content-item')
        boxoffice_element = driver.find_element(By.CSS_SELECTOR,'li[data-testid="title-boxoffice-cumulativeworldwidegross"] div.ipc-metadata-list-item__content-container ul.ipc-inline-list.ipc-inline-list--show-dividers.ipc-inline-list--inline.ipc-metadata-list-item__list-content.base li.ipc-inline-list__item span.ipc-metadata-list-item__list-content-item')
    except NoSuchElementException:
        pass
    budget_text = budget_element.text if budget_element else None
    budget.append(budget_text)
    boxoffice_text = boxoffice_element.text if boxoffice_element else None
    boxoffice.append(boxoffice_text)
    print('Box office:',boxoffice_text)

    # Lọc original_language
    original_language_element = None
    try:
        original_language_element = driver.find_element(By.CSS_SELECTOR,'li[data-testid="title-details-languages"] div.ipc-metadata-list-item__content-container ul.ipc-inline-list.ipc-inline-list--show-dividers.ipc-inline-list--inline.ipc-metadata-list-item__list-content.base li.ipc-inline-list__item a.ipc-metadata-list-item__list-content-item.ipc-metadata-list-item__list-content-item--link')
    except NoSuchElementException:
        pass
    original_language_text = original_language_element.text if original_language_element else None
    original_language.append(original_language_text)
    # Lọc companies
    companies_elements = None
    try:
        companies_elements = driver.find_elements(By.CSS_SELECTOR,'li[data-testid="title-details-companies"] div.ipc-metadata-list-item__content-container ul.ipc-inline-list.ipc-inline-list--show-dividers.ipc-inline-list--inline.ipc-metadata-list-item__list-content.base li')
    except NoSuchElementException:
        pass

    companies_text = ''
    for i, element in enumerate(companies_elements):
        if i < len(companies_elements) - 1:
            companies_text += element.text + ', '
        else:
            companies_text += element.text

    companies.append(companies_text)

    # Lọc genre
    movie_category_elements = None
    movie_category_elements = driver.find_elements(By.CSS_SELECTOR,'a.ipc-chip.ipc-chip--on-baseAlt span.ipc-chip__text')
    movie_category_text = ', '.join([element.text for element in movie_category_elements]) if movie_category_elements else None
    movie_category.append(movie_category_text)
    #Lọc director
    try:
        full_cast_crew_link = driver.find_element(By.CSS_SELECTOR, 'a[aria-label="See full cast and crew"]')
        driver.execute_script("arguments[0].click();", full_cast_crew_link)
        time.sleep(3)  # wait for the page to load

        director_tag_element = driver.find_element(By.CSS_SELECTOR, 'h4[id="director"]')
        table_element = director_tag_element.find_element(By.XPATH, './following-sibling::table[@class="simpleTable simpleCreditsTable"]')
        director_elements = table_element.find_elements(By.CSS_SELECTOR, 'td.name a[href^="/name/"]')
        directors_text = ', '.join([element.text for element in director_elements]) if director_elements else None
        directors.append(directors_text)
    except NoSuchElementException:
        directors.append(None)
    # Lọc Writing Credits
    writing_credits_tag_element = None
    try:
        writing_credits_tag_element = driver.find_element(By.CSS_SELECTOR,'h4[id="writer"]')
    except NoSuchElementException:
        pass

    if writing_credits_tag_element:
        table_element = writing_credits_tag_element.find_element(By.XPATH,'./following-sibling::table[@class="simpleTable simpleCreditsTable"]')
        writing_credits_element = table_element.find_elements(By.CSS_SELECTOR,'td.name a[href^="/name/"]')
    else:
        writing_credits_element = None

    writing_credits_text = ', '.join([element.text for element in writing_credits_element]) if writing_credits_element else None
    writing_credits.append(writing_credits_text)

# Clean up and close the driver
driver.quit()

Movies: The Outlaws
Ratings: 7.7
Certificate: TV-MA
Poster URL: https://m.media-amazon.com/images/M/MV5BNTMyZDJmOWQtZjUzYi00ZDE4LWExMGEtMzhiNDM0YTgxYmFiXkEyXkFqcGc@._V1_QL75_UX190_CR0,2,190,281_.jpg
Box office: None
Movies: Ánh Trăng
Ratings: 7.4
Certificate: R
Poster URL: https://m.media-amazon.com/images/M/MV5BNzQxNTIyODAxMV5BMl5BanBnXkFtZTgwNzQyMDA3OTE@._V1_QL75_UX190_CR0,0,190,281_.jpg
Box office: $65,172,611
Movies: Dracula: Quỷ Dữ Thức Tỉnh
Ratings: 6.1
Certificate: C18
Poster URL: https://m.media-amazon.com/images/M/MV5BMDg0YWViOWEtNDhhMC00NjRhLWE0OTctZWFiNGE0MDM3NjE4XkEyXkFqcGc@._V1_QL75_UY281_CR18,0,190,281_.jpg
Box office: $21,786,275
Movies: Shadowhunters: The Mortal Instruments
Ratings: 6.5
Certificate: TV-14
Poster URL: https://m.media-amazon.com/images/M/MV5BOTEzZThhZDYtZGFhMy00Y2Q1LWE2ZmMtNzg2MjYxY2I2NzIxXkEyXkFqcGc@._V1_QL75_UY281_CR1,0,190,281_.jpg
Box office: None
Movies: Weeds
Ratings: 7.9
Certificate: TV-MA
Poster URL: https://m.media-amazon.com/images/M/MV5BOWVmOGQ

In [ ]:
ratings

['7.7',
 '7.4',
 '6.1',
 '6.5',
 '7.9',
 '9.2',
 '6.4',
 '6.7',
 '7.4',
 '8.2',
 '7.1',
 '6.8',
 '6.8',
 '7.4',
 None,
 '6.6',
 '6.9',
 None,
 '7.2',
 '8.1',
 '7.3',
 '5.3',
 '7.9',
 None,
 '7.4',
 '6.7',
 None,
 '7.8',
 '7.2',
 '6.7',
 '8.3',
 '7.6',
 '8.5',
 '7.9',
 '5.7',
 '5.2',
 '6.2',
 '7.4',
 '7.0',
 '8.5',
 None,
 '7.6',
 '6.8',
 '6.9',
 '7.6',
 '6.7',
 '5.5',
 '8.3',
 '6.7',
 '7.1',
 '7.0',
 '7.5',
 '7.4',
 '5.0',
 '7.0',
 '8.2',
 '8.2',
 '7.1',
 '8.1',
 '7.6',
 '7.3',
 '5.7',
 '7.0',
 '8.7',
 '7.0',
 '5.3',
 '8.2',
 '7.5',
 '8.1',
 '6.8',
 '6.8',
 '6.2',
 '7.6',
 '7.7',
 '6.9',
 '7.7',
 '6.3',
 '6.4',
 '8.3',
 '8.7',
 '6.3',
 '6.0',
 '8.6',
 '7.7',
 '6.7',
 '7.2',
 '7.2',
 '7.6',
 '5.8',
 '7.7',
 '6.4',
 '7.6',
 '6.4',
 '8.1',
 '7.6',
 None,
 '7.7',
 '6.3',
 '8.0',
 None,
 '5.4',
 '7.5',
 '7.4',
 '8.1',
 '7.7',
 '7.1',
 '5.6',
 '8.4',
 '7.4',
 '7.9',
 '7.3',
 '7.3',
 '7.4',
 '8.5',
 '6.8',
 '6.6',
 '8.0',
 '7.4',
 '6.6',
 '6.3',
 '7.4',
 '6.7',
 '6.2',
 '3.9',
 '7.5',
 '8.4',

In [ ]:
# Define the expected length
expected_length = len(movie_links)

# Function to pad lists with None
def pad_list(lst, length):
    return lst + [None] * (length - len(lst))

# Pad all lists to the same length
movies = pad_list(movies, expected_length)
posters = pad_list(posters, expected_length)
film_actors = pad_list(film_actors, expected_length)
description = pad_list(description, expected_length)
duration = pad_list(duration, expected_length)
ratings = pad_list(ratings, expected_length)
release_date = pad_list(release_date, expected_length)
country = pad_list(country, expected_length)
budget = pad_list(budget, expected_length)
boxoffice = pad_list(boxoffice, expected_length)
certificate = pad_list(certificate, expected_length)
original_language = pad_list(original_language, expected_length)
companies = pad_list(companies, expected_length)
movie_category = pad_list(movie_category, expected_length)
directors = pad_list(directors, expected_length)
writing_credits = pad_list(writing_credits, expected_length)


In [ ]:
import pandas as pd

# Create a dictionary of the lists
data = {
    'Links' : movie_links,
    'Poster': posters,  # Thêm poster URL vào data
    'Movie': movies,
    'Actors': film_actors,
    'Duration': duration,
    'Ratings': ratings,
    'Release Date': release_date,
    'Country': country,
    'Budget': budget,
    'Box Office': boxoffice,
    'Certificate': certificate,
    'Original Language': original_language,
    'Companies': companies,
    'Genres': movie_category,
    'Directors': directors,
    'Writing Credits': writing_credits,
    'Descriptions': description
}

# Create the DataFrame
df = pd.DataFrame(data)

# Output the DataFrame to check

# Export the DataFrame to a CSV file
df.to_csv('IMDB_1001_1500.csv', index=False)

print("DataFrame successfully exported to movies_data.csv")


DataFrame successfully exported to movies_data.csv


In [ ]:
import pandas as pd
data = pd.read_csv('IMDB_1001_1500.csv')
data['Ratings'].isna().sum()

179

In [ ]:
data.isna().sum()

Links                  0
Poster                 2
Movie                  0
Actors                 1
Duration              28
Ratings              179
Release Date           1
Country                1
Budget               217
Box Office           230
Certificate           44
Original Language      6
Companies              5
Genres                 1
Directors              4
Writing Credits        9
Descriptions           0
dtype: int64

In [ ]:
data['Descriptions'].isna().sum()

0

In [ ]:
data['Ratings'].isna().sum()

179

In [ ]:
data

,Links,Poster,Movie,Actors,Duration,Ratings,Release Date,Country,Budget,Box Office,Certificate,Original Language,Companies,Genres,Directors,Writing Credits,Descriptions
0,https://www.imdb.com/title/tt11646832/?ref_=sr...,https://m.media-amazon.com/images/M/MV5BNTMyZD...,The Outlaws,"Rhianne Barreto, Darren Boyd, Gamba Cole, Jess...",58 minutes,7.7,"October 25, 2021 (United Kingdom)",United Kingdom,NaN,NaN,TV-MA,English,"Amazon Studios, BBC Studios, Big Talk Productions","Dark Comedy, Comedy, Crime, Drama, Thriller","John Butler, Alicia MacDonald, Stephen Merchan...","Elgin James, Elgin James, Stephen Merchant, St...",Seven strangers from different walks of life f...
1,https://www.imdb.com/title/tt4975722/?ref_=sr_t_2,https://m.media-amazon.com/images/M/MV5BNzQxNT...,Ánh Trăng,"Mahershala Ali, Naomie Harris, Trevante Rhodes...",1 hour 51 minutes,7.4,"November 18, 2016 (United States)",United States,"$4,000,000 (estimated)","$65,172,611",R,English,"A24, Plan B Entertainment, PASTEL","Coming-of-Age, Drama",Barry Jenkins,"Barry Jenkins, Tarell Alvin McCraney",A young African-American man grapples with his...
2,https://www.imdb.com/title/tt1001520/?ref_=sr_t_3,https://m.media-amazon.com/images/M/MV5BMDg0YW...,Dracula: Quỷ Dữ Thức Tỉnh,"Corey Hawkins, Aisling Franciosi, Liam Cunning...",1 hour 58 minutes,6.1,"August 18, 2023 (Vietnam)",United States,"$45,000,000 (estimated)","$21,786,275",C18,English,"Dreamworks Pictures, Reliance Entertainment, S...","Supernatural Horror, Vampire Horror, Fantasy, ...",André Øvredal,"Bram Stoker, Bragi F. Schut, Zak Olkewicz, Bra...",A crew sailing from Varna (Bulgaria) by the Bl...
3,https://www.imdb.com/title/tt4145054/?ref_=sr_t_4,https://m.media-amazon.com/images/M/MV5BOTEzZT...,Shadowhunters: The Mortal Instruments,"Katherine McNamara, Dominic Sherwood, Alberto ...",42 minutes,6.5,"January 12, 2016 (United States)",United States,NaN,NaN,TV-14,English,"Constantin Film, Don Carmody Television, Echel...","Superhero, Supernatural Fantasy, Teen Fantasy,...","Matthew Hastings, J. Miles Dale, Joshua Butler...","Cassandra Clare, Ed Decter, Ed Decter, Peter B...","After her mother disappears, Clary must ventur..."
4,https://www.imdb.com/title/tt0439100/?ref_=sr_t_5,https://m.media-amazon.com/images/M/MV5BOWVmOG...,Weeds,"Mary-Louise Parker, Hunter Parrish, Alexander ...",28 minutes,7.9,"August 7, 2005 (United States)",United States,NaN,NaN,TV-MA,English,"Lions Gate Television, Tilted Productions, Wee...","Dark Comedy, Satire, Stoner Comedy, Comedy, Cr...","Craig Zisk, Michael Trim, Scott Ellis, Julie A...","Jenji Kohan, Jenji Kohan, Victoria Morrow, Vic...",When a suburban mother turns to dealing mariju...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
495,https://www.imdb.com/title/tt14364876/?ref_=sr...,https://m.media-amazon.com/images/M/MV5BNDFiNT...,The Tower,"Gemma Whelan, Tahirah Sharif, Emmett J Scanlan...",NaN,NaN,"November 8, 2021 (United Kingdom)",United Kingdom,NaN,NaN,TV-14,English,"ITV - Independent Television, Mammoth Screen, ...","Crime, Drama, Mystery","Faye Gilbert, Rene van Pannevis, Jim Loach","Patrick Harbinson, Patrick Harbinson, Kate Lon...",After a long-serving cop and a teenage girl fa...
496,https://www.imdb.com/title/tt2557490/?ref_=sr_...,https://m.media-amazon.com/images/M/MV5BMTQ0ND...,Triệu Kiểu Chết Miền Viễn Tây,"Seth MacFarlane, Charlize Theron, Liam Neeson,...",1 hour 56 minutes,NaN,"July 18, 2014 (Vietnam)",United States,"$40,000,000 (estimated)","$87,189,756",C18,English,"Bluegrass Films, Fuzzy Door Productions, Media...","Parody, Slapstick, Comedy, Western",Seth MacFarlane,"Seth MacFarlane, Alec Sulkin, Wellesley Wild",As a cowardly farmer begins to fall for the my...
497,https://www.imdb.com/title/tt0091790/?ref_=sr_...,https://m.media-amazon.com/images/M/MV5BZWMxMT...,Pretty in Pink,"Molly Ringwald, Jon Cryer, Harry Dean Stanton,...",1 hour 37 minutes,NaN,"February 28, 1986 (United States)",United States,"$9,000,000 (estimated)","$40,480,6